Подготовка золотого стандарта для получения релевантных фрагментов для оценки базового пайплайна метриками

In [9]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics import precision_score, recall_score, ndcg_score
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from pipeline_file import DocumentationQA
import time

# Чтение данных из CSV
df = pd.read_csv('texts_with_answers.csv')

# Инициализация системы ретривала
qa_system = DocumentationQA()
qa_system.initialize_database()

# Получение API ключа
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

In [10]:
# Создаю список для хранения результатов
retrieval_results = []

# Обрабатываю каждый вопрос
for index, row in df.iterrows():
    question = row['question']
    fragments = qa_system.search_similar_paragraphs(question, top_k=6)
    retrieval_results.append({
        'question': question,
        'fragments': fragments
    })

In [11]:
def evaluate_relevance_with_claude(question, fragment_text, api_key):
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": 'chad-ea4e58bc0ac4441ca91e1188ca33120cpsekgbal'
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

# Добавляю оценки релевантности к результатам
for result in retrieval_results:
    question = result['question']
    for i, (text, name, score) in enumerate(result['fragments']):
        # Делаю задержку между запросами, чтобы не превысить лимиты API
        time.sleep(2)  # Увеличила задержку до 2 секунд для надежности
        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        result['fragments'][i] = (text, name, score, relevance_score)

In [14]:
def calculate_metrics(retrieval_results):
    metrics = {
        'recall@1': [],
        'recall@4': [],
        'recall@6': [],
        'precision@1': [],
        'precision@4': [],
        'precision@6': [],
        'mrr@4': [],
        'mrr@6': [],
        'ndcg@4': [],
        'ndcg@6': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Сортировка фрагментов по скору из системы ретривала (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[2], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[3] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@1
            relevant_at_1 = sum(1 for f in retrieved_fragments[:1] if f[3] >= 4)
            metrics['recall@1'].append(relevant_at_1 / total_relevant)
            
            # Recall@4
            relevant_at_4 = sum(1 for f in retrieved_fragments[:4] if f[3] >= 4)
            metrics['recall@4'].append(relevant_at_4 / total_relevant)
            
            # Recall@6
            relevant_at_6 = sum(1 for f in retrieved_fragments[:6] if f[3] >= 4)
            metrics['recall@6'].append(relevant_at_6 / total_relevant)
            
            # Precision@1
            metrics['precision@1'].append(relevant_at_1 / 1)
            
            # Precision@4
            metrics['precision@4'].append(relevant_at_4 / 4)  # Исправлено с 3 на 4
            
            # Precision@6
            metrics['precision@6'].append(relevant_at_6 / 6)
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics['recall@1'].append(1.0)
            metrics['recall@4'].append(1.0)
            metrics['recall@6'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics['precision@1'].append(0.0)
            metrics['precision@4'].append(0.0)
            metrics['precision@6'].append(0.0)
        
        # MRR@4 (Mean Reciprocal Rank для первых 4)
        first_relevant_rank_at_4 = next((i + 1 for i, f in enumerate(retrieved_fragments[:4]) if f[3] >= 4), 0)
        if first_relevant_rank_at_4 > 0:
            metrics['mrr@4'].append(1.0 / first_relevant_rank_at_4)
        else:
            metrics['mrr@4'].append(0.0)
            
        # MRR@6 (Mean Reciprocal Rank для первых 6)
        first_relevant_rank_at_6 = next((i + 1 for i, f in enumerate(retrieved_fragments[:6]) if f[3] >= 4), 0)
        if first_relevant_rank_at_6 > 0:
            metrics['mrr@6'].append(1.0 / first_relevant_rank_at_6)
        else:
            metrics['mrr@6'].append(0.0)
        
        # nDCG@4
        if len(sorted_fragments) >= 4 and len(retrieved_fragments) >= 4:
            # Берем только первые 4 документа для nDCG@4
            true_relevance_4 = np.array([f[3] for f in sorted_fragments[:4]])
            predicted_relevance_4 = np.array([f[3] for f in retrieved_fragments[:4]])
            
            try:
                ndcg_4 = ndcg_score([true_relevance_4], [predicted_relevance_4])
                metrics['ndcg@4'].append(ndcg_4)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@4: {e}")
                metrics['ndcg@4'].append(0.0)
        else:
            metrics['ndcg@4'].append(0.0)
            
        # nDCG@6
        if len(sorted_fragments) >= 6 and len(retrieved_fragments) >= 6:
            # Берем только первые 6 документов для nDCG@6
            true_relevance_6 = np.array([f[3] for f in sorted_fragments[:6]])
            predicted_relevance_6 = np.array([f[3] for f in retrieved_fragments[:6]])
            
            try:
                ndcg_6 = ndcg_score([true_relevance_6], [predicted_relevance_6])
                metrics['ndcg@6'].append(ndcg_6)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@6: {e}")
                metrics['ndcg@6'].append(0.0)
        else:
            metrics['ndcg@6'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Вычисляем метрики
metrics_results = calculate_metrics(retrieval_results)

In [16]:
# Вывод результатов
print("Оценка подхода к ретривалу:")
print(f"Recall@1: {metrics_results['recall@1']:.4f}")
print(f"Recall@4: {metrics_results['recall@4']:.4f}")
print(f"Recall@6: {metrics_results['recall@6']:.4f}")
print(f"Precision@1: {metrics_results['precision@1']:.4f}")
print(f"Precision@4: {metrics_results['precision@4']:.4f}")
print(f"Precision@6: {metrics_results['precision@6']:.4f}")
print(f"MRR@4: {metrics_results['mrr@4']:.4f}")
print(f"MRR@6: {metrics_results['mrr@6']:.4f}")
print(f"nDCG@4: {metrics_results['ndcg@4']:.4f}")
print(f"nDCG@6: {metrics_results['ndcg@6']:.4f}")

Оценка подхода к ретривалу:
Recall@1: 0.4440
Recall@4: 0.8444
Recall@6: 1.0000
Precision@1: 0.7167
Precision@4: 0.4125
Precision@6: 0.3431
MRR@4: 0.8014
MRR@6: 0.8092
nDCG@4: 0.9626
nDCG@6: 0.9467


In [2]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics import precision_score, recall_score, ndcg_score
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from pipeline_file import DocumentationQA
import time

# Чтение данных из CSV
df = pd.read_csv('texts_with_answers.csv')

# Инициализация системы ретривала
qa_system = DocumentationQA()
qa_system.initialize_database()

# Инициализация reranker'а
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

# Получение API ключа
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

# Создаю списки для хранения результатов
retrieval_results_4 = []  # Результаты для top-4 после reranker'а
retrieval_results_6 = []  # Результаты для top-6 после reranker'а

# Обрабатываю каждый вопрос
for index, row in df.iterrows():
    question = row['question']
    
    # Сначала получаем 20 фрагментов
    initial_fragments = qa_system.search_similar_paragraphs(question, top_k=20)
    
    # Подготовка пар "вопрос-фрагмент" для reranker'а
    pairs = [(question, fragment[0]) for fragment in initial_fragments]
    
    # Применение reranker'а
    reranker_scores = reranker.predict(pairs)
    
    # Создаем кортежи (текст, имя, первоначальный скор, скор reranker'а)
    reranked_fragments = [(fragment[0], fragment[1], fragment[2], score) 
                         for fragment, score in zip(initial_fragments, reranker_scores)]
    
    # Сортировка по скору reranker'а
    reranked_fragments = sorted(reranked_fragments, key=lambda x: x[3], reverse=True)
    
    # Отбираем top-4 и top-6 после reranking'а
    top_4_fragments = reranked_fragments[:4]
    top_6_fragments = reranked_fragments[:6]
    
    # Сохраняем результаты
    retrieval_results_4.append({
        'question': question,
        'fragments': top_4_fragments
    })
    
    retrieval_results_6.append({
        'question': question,
        'fragments': top_6_fragments
    })

def evaluate_relevance_with_claude(question, fragment_text, api_key):
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": 'chad-ea4e58bc0ac4441ca91e1188ca33120cpsekgbal'
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

# Добавляю оценки релевантности к результатам для top-4
for result in retrieval_results_4:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Делаю задержку между запросами
        time.sleep(2)
        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

# Добавляю оценки релевантности к результатам для top-6
for result in retrieval_results_6:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Для фрагментов, которые уже оценены в top-4, не делаем повторный запрос
        if i < 4:
            # Находим соответствующий фрагмент в top-4 и берем оценку оттуда
            for r4 in retrieval_results_4:
                if r4['question'] == question:
                    # Ищем совпадающий фрагмент
                    for f4 in r4['fragments']:
                        if f4[0] == text:
                            relevance_score = f4[4]  # Используем уже полученную оценку
                            break
                    break
        else:
            # Для новых фрагментов делаем запрос
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

def calculate_metrics_reranker(retrieval_results, k_value):
    metrics = {
        f'recall@{k_value}': [],
        f'precision@{k_value}': [],
        f'mrr@{k_value}': [],
        f'ndcg@{k_value}': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[4], reverse=True)
        
        # Сортировка фрагментов по скору reranker'а (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[4] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@k
            relevant_at_k = sum(1 for f in retrieved_fragments[:k_value] if f[4] >= 4)
            metrics[f'recall@{k_value}'].append(relevant_at_k / total_relevant)
            
            # Precision@k
            metrics[f'precision@{k_value}'].append(relevant_at_k / k_value)
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics[f'recall@{k_value}'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics[f'precision@{k_value}'].append(0.0)
        
        # MRR@k (Mean Reciprocal Rank)
        first_relevant_rank = next((i + 1 for i, f in enumerate(retrieved_fragments[:k_value]) if f[4] >= 4), 0)
        if first_relevant_rank > 0:
            metrics[f'mrr@{k_value}'].append(1.0 / first_relevant_rank)
        else:
            metrics[f'mrr@{k_value}'].append(0.0)
        
        # nDCG@k
        if len(sorted_fragments) >= k_value and len(retrieved_fragments) >= k_value:
            # Берем только первые k документов
            true_relevance = np.array([f[4] for f in sorted_fragments[:k_value]])
            predicted_relevance = np.array([f[4] for f in retrieved_fragments[:k_value]])
            
            try:
                ndcg = ndcg_score([true_relevance], [predicted_relevance])
                metrics[f'ndcg@{k_value}'].append(ndcg)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@{k_value}: {e}")
                metrics[f'ndcg@{k_value}'].append(0.0)
        else:
            metrics[f'ndcg@{k_value}'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Вычисляем метрики для top-4 результатов
metrics_results_4 = calculate_metrics_reranker(retrieval_results_4, 4)

# Вычисляем метрики для top-6 результатов
metrics_results_6 = calculate_metrics_reranker(retrieval_results_6, 6)

# Вывод результатов
print("Оценка подхода с reranker и top-4 фрагментами:")
print(f"Recall@4: {metrics_results_4['recall@4']:.4f}")
print(f"Precision@4: {metrics_results_4['precision@4']:.4f}")
print(f"MRR@4: {metrics_results_4['mrr@4']:.4f}")
print(f"nDCG@4: {metrics_results_4['ndcg@4']:.4f}")

print("\nОценка подхода с reranker и top-6 фрагментами:")
print(f"Recall@6: {metrics_results_6['recall@6']:.4f}")
print(f"Precision@6: {metrics_results_6['precision@6']:.4f}")
print(f"MRR@6: {metrics_results_6['mrr@6']:.4f}")
print(f"nDCG@6: {metrics_results_6['ndcg@6']:.4f}")

Оценка подхода с reranker и top-4 фрагментами:
Recall@4: 1.0000
Precision@4: 0.4604
MRR@4: 0.8326
nDCG@4: 0.9668

Оценка подхода с reranker и top-6 фрагментами:
Recall@6: 1.0000
Precision@6: 0.3708
MRR@6: 0.8360
nDCG@6: 0.9608


In [2]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics import precision_score, recall_score, ndcg_score
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from pipeline_file import DocumentationQA
import time
from FlagEmbedding import FlagReranker

# Чтение данных из CSV
df = pd.read_csv('texts_with_answers.csv')

# Инициализация системы ретривала
qa_system = DocumentationQA()
qa_system.initialize_database()

# Инициализация reranker'а
reranker = FlagReranker('BAAI/bge-reranker-base')

# Получение API ключа
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

# Создаю списки для хранения результатов
retrieval_results_4 = []  # Результаты для top-4 после reranker'а
retrieval_results_6 = []  # Результаты для top-6 после reranker'а

# Обрабатываю каждый вопрос
for index, row in df.iterrows():
    question = row['question']
    
    # Сначала получаем 20 фрагментов
    initial_fragments = qa_system.search_similar_paragraphs(question, top_k=20)
    
    # Подготовка пар "вопрос-фрагмент" для reranker'а
    pairs = [(question, fragment[0]) for fragment in initial_fragments]
    
    # Применение reranker'а
    reranker_scores = reranker.compute_score(pairs)
    
    # Создаем кортежи (текст, имя, первоначальный скор, скор reranker'а)
    reranked_fragments = [(fragment[0], fragment[1], fragment[2], score) 
                         for fragment, score in zip(initial_fragments, reranker_scores)]
    
    # Сортировка по скору reranker'а
    reranked_fragments = sorted(reranked_fragments, key=lambda x: x[3], reverse=True)
    
    # Отбираем top-4 и top-6 после reranking'а
    top_4_fragments = reranked_fragments[:4]
    top_6_fragments = reranked_fragments[:6]
    
    # Сохраняем результаты
    retrieval_results_4.append({
        'question': question,
        'fragments': top_4_fragments
    })
    
    retrieval_results_6.append({
        'question': question,
        'fragments': top_6_fragments
    })

def evaluate_relevance_with_claude(question, fragment_text, api_key):
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": 'chad-ea4e58bc0ac4441ca91e1188ca33120cpsekgbal'
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

# Добавляю оценки релевантности к результатам для top-4
for result in retrieval_results_4:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Делаю задержку между запросами
        time.sleep(2)
        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

# Добавляю оценки релевантности к результатам для top-6
for result in retrieval_results_6:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Для фрагментов, которые уже оценены в top-4, не делаем повторный запрос
        if i < 4:
            # Находим соответствующий фрагмент в top-4 и берем оценку оттуда
            for r4 in retrieval_results_4:
                if r4['question'] == question:
                    # Ищем совпадающий фрагмент
                    for f4 in r4['fragments']:
                        if f4[0] == text:
                            relevance_score = f4[4]  # Используем уже полученную оценку
                            break
                    break
        else:
            # Для новых фрагментов делаем запрос
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

def calculate_metrics_reranker(retrieval_results, k_value):
    metrics = {
        f'recall@{k_value}': [],
        f'precision@{k_value}': [],
        f'mrr@{k_value}': [],
        f'ndcg@{k_value}': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[4], reverse=True)
        
        # Сортировка фрагментов по скору reranker'а (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[4] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@k
            relevant_at_k = sum(1 for f in retrieved_fragments[:k_value] if f[4] >= 4)
            metrics[f'recall@{k_value}'].append(relevant_at_k / total_relevant)
            
            # Precision@k
            metrics[f'precision@{k_value}'].append(relevant_at_k / k_value)
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics[f'recall@{k_value}'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics[f'precision@{k_value}'].append(0.0)
        
        # MRR@k (Mean Reciprocal Rank)
        first_relevant_rank = next((i + 1 for i, f in enumerate(retrieved_fragments[:k_value]) if f[4] >= 4), 0)
        if first_relevant_rank > 0:
            metrics[f'mrr@{k_value}'].append(1.0 / first_relevant_rank)
        else:
            metrics[f'mrr@{k_value}'].append(0.0)
        
        # nDCG@k
        if len(sorted_fragments) >= k_value and len(retrieved_fragments) >= k_value:
            # Берем только первые k документов
            true_relevance = np.array([f[4] for f in sorted_fragments[:k_value]])
            predicted_relevance = np.array([f[4] for f in retrieved_fragments[:k_value]])
            
            try:
                ndcg = ndcg_score([true_relevance], [predicted_relevance])
                metrics[f'ndcg@{k_value}'].append(ndcg)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@{k_value}: {e}")
                metrics[f'ndcg@{k_value}'].append(0.0)
        else:
            metrics[f'ndcg@{k_value}'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Вычисляем метрики для top-4 результатов
metrics_results_4 = calculate_metrics_reranker(retrieval_results_4, 4)

# Вычисляем метрики для top-6 результатов
metrics_results_6 = calculate_metrics_reranker(retrieval_results_6, 6)

# Вывод результатов
print("Оценка подхода с reranker и top-4 фрагментами:")
print(f"Recall@4: {metrics_results_4['recall@4']:.4f}")
print(f"Precision@4: {metrics_results_4['precision@4']:.4f}")
print(f"MRR@4: {metrics_results_4['mrr@4']:.4f}")
print(f"nDCG@4: {metrics_results_4['ndcg@4']:.4f}")

print("\nОценка подхода с reranker и top-6 фрагментами:")
print(f"Recall@6: {metrics_results_6['recall@6']:.4f}")
print(f"Precision@6: {metrics_results_6['precision@6']:.4f}")
print(f"MRR@6: {metrics_results_6['mrr@6']:.4f}")
print(f"nDCG@6: {metrics_results_6['ndcg@6']:.4f}")

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Оценка подхода с reranker и top-4 фрагментами:
Recall@4: 1.0000
Precision@4: 0.4625
MRR@4: 0.8618
nDCG@4: 0.9707

Оценка подхода с reranker и top-6 фрагментами:
Recall@6: 1.0000
Precision@6: 0.3806
MRR@6: 0.8646
nDCG@6: 0.9602


In [6]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics import precision_score, recall_score, ndcg_score
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from pipeline_file import DocumentationQA
import time
from FlagEmbedding import FlagReranker
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Инициализация reranker'а
class TinyBERTReranker:
    def __init__(self, model_name='nboost/pt-tinybert-msmarco', batch_size=16, device=None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        
        # Выбор устройства
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        
        self.batch_size = batch_size
    
    def compute_score(self, pairs):
        scores = []
        for i in range(0, len(pairs), self.batch_size):
            batch_pairs = pairs[i:i+self.batch_size]
            inputs = self.tokenizer(
                batch_pairs,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=512
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.model(**inputs)
                batch_scores = torch.nn.functional.softmax(outputs.logits, dim=1)[:, 1].cpu().numpy()
                scores.extend(batch_scores)
        return scores

# Чтение данных из CSV
df = pd.read_csv('texts_with_answers.csv')

# Инициализация системы ретривала
qa_system = DocumentationQA()
qa_system.initialize_database()

# Инициализация reranker'а
reranker = TinyBERTReranker()

# Получение API ключа
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

# Создаю списки для хранения результатов
retrieval_results_4 = []  # Результаты для top-4 после reranker'а
retrieval_results_6 = []  # Результаты для top-6 после reranker'а

# Обрабатываю каждый вопрос
for index, row in df.iterrows():
    question = row['question']
    
    # Сначала получаем 20 фрагментов
    initial_fragments = qa_system.search_similar_paragraphs(question, top_k=20)
    
    # Подготовка пар "вопрос-фрагмент" для reranker'а
    pairs = [(question, fragment[0]) for fragment in initial_fragments]
    
    # Применение reranker'а
    reranker_scores = reranker.compute_score(pairs)
    
    # Создаем кортежи (текст, имя, первоначальный скор, скор reranker'а)
    reranked_fragments = [(fragment[0], fragment[1], fragment[2], score) 
                         for fragment, score in zip(initial_fragments, reranker_scores)]
    
    # Сортировка по скору reranker'а
    reranked_fragments = sorted(reranked_fragments, key=lambda x: x[3], reverse=True)
    
    # Отбираем top-4 и top-6 после reranking'а
    top_4_fragments = reranked_fragments[:4]
    top_6_fragments = reranked_fragments[:6]
    
    # Сохраняем результаты
    retrieval_results_4.append({
        'question': question,
        'fragments': top_4_fragments
    })
    
    retrieval_results_6.append({
        'question': question,
        'fragments': top_6_fragments
    })

def evaluate_relevance_with_claude(question, fragment_text, api_key):
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": 'chad-ea4e58bc0ac4441ca91e1188ca33120cpsekgbal'
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

# Добавляю оценки релевантности к результатам для top-4
for result in retrieval_results_4:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Делаю задержку между запросами
        time.sleep(2)
        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

# Добавляю оценки релевантности к результатам для top-6
for result in retrieval_results_6:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Для фрагментов, которые уже оценены в top-4, не делаем повторный запрос
        if i < 4:
            # Находим соответствующий фрагмент в top-4 и берем оценку оттуда
            for r4 in retrieval_results_4:
                if r4['question'] == question:
                    # Ищем совпадающий фрагмент
                    for f4 in r4['fragments']:
                        if f4[0] == text:
                            relevance_score = f4[4]  # Используем уже полученную оценку
                            break
                    break
        else:
            # Для новых фрагментов делаем запрос
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

def calculate_metrics_reranker(retrieval_results, k_value):
    metrics = {
        f'recall@{k_value}': [],
        f'precision@{k_value}': [],
        f'mrr@{k_value}': [],
        f'ndcg@{k_value}': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[4], reverse=True)
        
        # Сортировка фрагментов по скору reranker'а (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[4] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@k
            relevant_at_k = sum(1 for f in retrieved_fragments[:k_value] if f[4] >= 4)
            metrics[f'recall@{k_value}'].append(relevant_at_k / total_relevant)
            
            # Precision@k
            metrics[f'precision@{k_value}'].append(relevant_at_k / k_value)
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics[f'recall@{k_value}'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics[f'precision@{k_value}'].append(0.0)
        
        # MRR@k (Mean Reciprocal Rank)
        first_relevant_rank = next((i + 1 for i, f in enumerate(retrieved_fragments[:k_value]) if f[4] >= 4), 0)
        if first_relevant_rank > 0:
            metrics[f'mrr@{k_value}'].append(1.0 / first_relevant_rank)
        else:
            metrics[f'mrr@{k_value}'].append(0.0)
        
        # nDCG@k
        if len(sorted_fragments) >= k_value and len(retrieved_fragments) >= k_value:
            # Берем только первые k документов
            true_relevance = np.array([f[4] for f in sorted_fragments[:k_value]])
            predicted_relevance = np.array([f[4] for f in retrieved_fragments[:k_value]])
            
            try:
                ndcg = ndcg_score([true_relevance], [predicted_relevance])
                metrics[f'ndcg@{k_value}'].append(ndcg)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@{k_value}: {e}")
                metrics[f'ndcg@{k_value}'].append(0.0)
        else:
            metrics[f'ndcg@{k_value}'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Вычисляем метрики для top-4 результатов
metrics_results_4 = calculate_metrics_reranker(retrieval_results_4, 4)

# Вычисляем метрики для top-6 результатов
metrics_results_6 = calculate_metrics_reranker(retrieval_results_6, 6)

# Вывод результатов
print("Оценка подхода с reranker и top-4 фрагментами:")
print(f"Recall@4: {metrics_results_4['recall@4']:.4f}")
print(f"Precision@4: {metrics_results_4['precision@4']:.4f}")
print(f"MRR@4: {metrics_results_4['mrr@4']:.4f}")
print(f"nDCG@4: {metrics_results_4['ndcg@4']:.4f}")

print("\nОценка подхода с reranker и top-6 фрагментами:")
print(f"Recall@6: {metrics_results_6['recall@6']:.4f}")
print(f"Precision@6: {metrics_results_6['precision@6']:.4f}")
print(f"MRR@6: {metrics_results_6['mrr@6']:.4f}")
print(f"nDCG@6: {metrics_results_6['ndcg@6']:.4f}")

config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

C:\Users\sekho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sekho\.cache\huggingface\hub\models--nboost--pt-tinybert-msmarco. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/58.4M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/58.4M [00:00<?, ?B/s]

Оценка подхода с reranker и top-4 фрагментами:
Recall@4: 1.0000
Precision@4: 0.3729
MRR@4: 0.6813
nDCG@4: 0.9473

Оценка подхода с reranker и top-6 фрагментами:
Recall@6: 1.0000
Precision@6: 0.3069
MRR@6: 0.6890
nDCG@6: 0.9401


In [7]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.metrics import precision_score, recall_score, ndcg_score
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from pipeline_file import DocumentationQA
import time
from FlagEmbedding import FlagReranker

# Чтение данных из CSV
df = pd.read_csv('texts_with_answers.csv')

# Инициализация системы ретривала
qa_system = DocumentationQA()
qa_system.initialize_database()

# Инициализация reranker'а
reranker = CrossEncoder('sentence-transformers/msmarco-distilbert-base-tas-b')

# Получение API ключа
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

# Создаю списки для хранения результатов
retrieval_results_4 = []  # Результаты для top-4 после reranker'а
retrieval_results_6 = []  # Результаты для top-6 после reranker'а

# Обрабатываю каждый вопрос
for index, row in df.iterrows():
    question = row['question']
    
    # Сначала получаем 20 фрагментов
    initial_fragments = qa_system.search_similar_paragraphs(question, top_k=20)
    
    # Подготовка пар "вопрос-фрагмент" для reranker'а
    pairs = [(question, fragment[0]) for fragment in initial_fragments]
    
    # Применение reranker'а
    reranker_scores = reranker.predict(pairs, batch_size=16, show_progress_bar=True, convert_to_numpy=True)
    
    # Создаем кортежи (текст, имя, первоначальный скор, скор reranker'а)
    reranked_fragments = [(fragment[0], fragment[1], fragment[2], score) 
                         for fragment, score in zip(initial_fragments, reranker_scores)]
    
    # Сортировка по скору reranker'а
    reranked_fragments = sorted(reranked_fragments, key=lambda x: x[3], reverse=True)
    
    # Отбираем top-4 и top-6 после reranking'а
    top_4_fragments = reranked_fragments[:4]
    top_6_fragments = reranked_fragments[:6]
    
    # Сохраняем результаты
    retrieval_results_4.append({
        'question': question,
        'fragments': top_4_fragments
    })
    
    retrieval_results_6.append({
        'question': question,
        'fragments': top_6_fragments
    })

def evaluate_relevance_with_claude(question, fragment_text, api_key):
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    prompt = f"""
    Задача: оценить релевантность текстового фрагмента вопросу.
    
    Вопрос: {question}
    
    Фрагмент: {fragment_text}
    
    Оцени релевантность фрагмента к вопросу по шкале от 1 до 5, где:
    1 - совершенно не релевантен
    2 - слабо релевантен
    3 - умеренно релевантен
    4 - очень релевантен
    5 - идеально релевантен
    
    Ответь только числом от 1 до 5 без пояснений.
    """
    
    # Формируем запрос согласно примеру
    request_json = {
        "message": prompt,
        "api_key": 'chad-ea4e58bc0ac4441ca91e1188ca33120cpsekgbal'
    }
    
    try:
        # Отправляем запрос и дожидаемся ответа
        response = requests.post(url=url, json=request_json)
        
        # Проверяем, отправился ли запрос
        if response.status_code != 200:
            print(f'Ошибка! Код http-ответа: {response.status_code}')
            return 1  # Возвращаем минимальную оценку в случае ошибки
        else:
            # Получаем текст ответа и преобразовываем в dict
            resp_json = response.json()
            
            # Если успешен ответ, то извлекаем результат
            if resp_json['is_success']:
                resp_msg = resp_json['response'].strip()
                # Ищем число от 1 до 5 в ответе
                import re
                score_match = re.search(r'[1-5]', resp_msg)
                if score_match:
                    relevance_score = int(score_match.group(0))
                    return relevance_score
                else:
                    print(f'Не удалось извлечь оценку из ответа: {resp_msg}')
                    return 3  # Средняя оценка по умолчанию в случае неоднозначного ответа
            else:
                error = resp_json['error_message']
                print(f'Ошибка: {error}')
                return 1  # Возвращаем минимальную оценку в случае ошибки
    except Exception as e:
        print(f'Исключение при обработке запроса: {str(e)}')
        return 1  # Возвращаем минимальную оценку в случае ошибки

# Добавляю оценки релевантности к результатам для top-4
for result in retrieval_results_4:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Делаю задержку между запросами
        time.sleep(2)
        relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

# Добавляю оценки релевантности к результатам для top-6
for result in retrieval_results_6:
    question = result['question']
    for i, (text, name, initial_score, reranker_score) in enumerate(result['fragments']):
        # Для фрагментов, которые уже оценены в top-4, не делаем повторный запрос
        if i < 4:
            # Находим соответствующий фрагмент в top-4 и берем оценку оттуда
            for r4 in retrieval_results_4:
                if r4['question'] == question:
                    # Ищем совпадающий фрагмент
                    for f4 in r4['fragments']:
                        if f4[0] == text:
                            relevance_score = f4[4]  # Используем уже полученную оценку
                            break
                    break
        else:
            # Для новых фрагментов делаем запрос
            time.sleep(2)
            relevance_score = evaluate_relevance_with_claude(question, text, api_key)
        
        result['fragments'][i] = (text, name, initial_score, reranker_score, relevance_score)

def calculate_metrics_reranker(retrieval_results, k_value):
    metrics = {
        f'recall@{k_value}': [],
        f'precision@{k_value}': [],
        f'mrr@{k_value}': [],
        f'ndcg@{k_value}': []
    }
    
    for result in retrieval_results:
        fragments = result['fragments']
        
        # Сортировка фрагментов по оценке релевантности от Claude (по убыванию)
        sorted_fragments = sorted(fragments, key=lambda x: x[4], reverse=True)
        
        # Сортировка фрагментов по скору reranker'а (по убыванию)
        retrieved_fragments = sorted(fragments, key=lambda x: x[3], reverse=True)
        
        # Вычисление Recall@k
        relevant_fragments = [f for f in sorted_fragments if f[4] >= 4]  # Считаем релевантными фрагменты с оценкой >= 4
        total_relevant = len(relevant_fragments)
        
        if total_relevant > 0:
            # Recall@k
            relevant_at_k = sum(1 for f in retrieved_fragments[:k_value] if f[4] >= 4)
            metrics[f'recall@{k_value}'].append(relevant_at_k / total_relevant)
            
            # Precision@k
            metrics[f'precision@{k_value}'].append(relevant_at_k / k_value)
        else:
            # Если нет релевантных фрагментов, устанавливаем recall = 1.0 (все релевантные найдены)
            metrics[f'recall@{k_value}'].append(1.0)
            
            # Если нет релевантных фрагментов, устанавливаем precision = 0.0
            metrics[f'precision@{k_value}'].append(0.0)
        
        # MRR@k (Mean Reciprocal Rank)
        first_relevant_rank = next((i + 1 for i, f in enumerate(retrieved_fragments[:k_value]) if f[4] >= 4), 0)
        if first_relevant_rank > 0:
            metrics[f'mrr@{k_value}'].append(1.0 / first_relevant_rank)
        else:
            metrics[f'mrr@{k_value}'].append(0.0)
        
        # nDCG@k
        if len(sorted_fragments) >= k_value and len(retrieved_fragments) >= k_value:
            # Берем только первые k документов
            true_relevance = np.array([f[4] for f in sorted_fragments[:k_value]])
            predicted_relevance = np.array([f[4] for f in retrieved_fragments[:k_value]])
            
            try:
                ndcg = ndcg_score([true_relevance], [predicted_relevance])
                metrics[f'ndcg@{k_value}'].append(ndcg)
            except Exception as e:
                print(f"Ошибка при вычислении nDCG@{k_value}: {e}")
                metrics[f'ndcg@{k_value}'].append(0.0)
        else:
            metrics[f'ndcg@{k_value}'].append(0.0)
    
    # Вычисляем средние значения метрик
    result_metrics = {}
    for key, values in metrics.items():
        result_metrics[key] = sum(values) / len(values) if values else 0.0
    
    return result_metrics

# Вычисляем метрики для top-4 результатов
metrics_results_4 = calculate_metrics_reranker(retrieval_results_4, 4)

# Вычисляем метрики для top-6 результатов
metrics_results_6 = calculate_metrics_reranker(retrieval_results_6, 6)

# Вывод результатов
print("Оценка подхода с reranker и top-4 фрагментами:")
print(f"Recall@4: {metrics_results_4['recall@4']:.4f}")
print(f"Precision@4: {metrics_results_4['precision@4']:.4f}")
print(f"MRR@4: {metrics_results_4['mrr@4']:.4f}")
print(f"nDCG@4: {metrics_results_4['ndcg@4']:.4f}")

print("\nОценка подхода с reranker и top-6 фрагментами:")
print(f"Recall@6: {metrics_results_6['recall@6']:.4f}")
print(f"Precision@6: {metrics_results_6['precision@6']:.4f}")
print(f"MRR@6: {metrics_results_6['mrr@6']:.4f}")
print(f"nDCG@6: {metrics_results_6['ndcg@6']:.4f}")

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

C:\Users\sekho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sekho\.cache\huggingface\hub\models--sentence-transformers--msmarco-distilbert-base-tas-b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(messa

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-tas-b and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Оценка подхода с reranker и top-4 фрагментами:
Recall@4: 1.0000
Precision@4: 0.2083
MRR@4: 0.3069
nDCG@4: 0.9244

Оценка подхода с reranker и top-6 фрагментами:
Recall@6: 1.0000
Precision@6: 0.2028
MRR@6: 0.3286
nDCG@6: 0.9084
